# Testing

In [1]:
library(data.table)

In [6]:
# read data files
p = fread("../models/MobHealthRecycling/output/testing/parameters-1-1.csv")
m = fread("../models/MobHealthRecycling/output/testing/mortality-1-1.csv")
c = fread("../models/MobHealthRecycling/output/testing/county-1-1.csv")


In [9]:
summary(c$model_time)


   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
     60     315     570     570     825    1080 

In [11]:
summary(c$rank_slope)


      Min.    1st Qu.     Median       Mean    3rd Qu.       Max. 
-0.2172636 -0.0464756  0.0000000  0.0006927  0.0451803  0.2619711 